In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score

In [3]:
data_train = pd.read_csv('../train_data_n.csv')
features_train = data_train.drop(['target'], axis=1)
target_train = data_train['target']

data_test = pd.read_csv('../test_data_n.csv')
features_test = data_test.drop(['target'], axis=1)
target_test = data_test['target']

In [4]:
class SGDLinearRegression:
    def __init__(self, step_size, epochs, batch_size, reg_weight):
        self.step_size = step_size
        self.epochs = epochs
        self.batch_size = batch_size
        self.reg_weight = reg_weight

    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.zeros(X.shape[1])

        for _ in range(self.epochs):
            batches_count = X.shape[0] // self.batch_size
            for i in range(batches_count):
                begin = i * self.batch_size
                end = (i + 1) * self.batch_size
                X_batch = X[begin:end, :]
                y_batch = y[begin:end]

                gradient = 2 * X_batch.T.dot(X_batch.dot(w) - y_batch) / X_batch.shape[0]
                # копируем вектор w, чтобы его не менять
                reg = 2 * w.copy()
                reg[0] = 0
                gradient += self.reg_weight * reg
                w -= self.step_size * gradient

        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

In [7]:
for reg_weight in [0.0, 0.1, 1.0, 10.0]:
    print("Регуляризация:", reg_weight)
    model = SGDLinearRegression(0.01, 10, 100, reg_weight)
    model.fit(features_train, target_train)
    pred_train = model.predict(features_train)
    pred_test = model.predict(features_test)
    print(r2_score(target_train, pred_train))
    print(r2_score(target_test, pred_test))

Регуляризация: 0.0
0.21882420870125097
0.06295838397587328
Регуляризация: 0.1
0.21488138182726813
0.07001298159200142
Регуляризация: 1.0
0.16660956844692543
0.08061382017133167
Регуляризация: 10.0
0.03944665027115968
0.024121773187080975
